## 有向图

### 邻接表表示法

In [15]:
class DirectedGraph():
    def __init__(self,d):
        if isinstance(d,dict):
            self.__graph = d
        else:
            self.__graph = dict()
            print('Sth error')
            
    def __generatePath(self,graph,path,end,results):
        current = path[-1]
        if current == end:
            results.append(path)
        else:
            for n in graph[current]:
                if n not in path:
                    self.__generatePath(graph,path+[n],end,results)
    
    def search(self,start,end):
        self.__results = []
        self.__generatePath(self.__graph,[start],end,self.__results)
        self.__results.sort(key = lambda x:len(x))
        print('The path from',self.__results[0][0],'to',self.__results[0][-1],'is:')
        for path in self.__results:
            print(path)

In [16]:
d = {'A':['B','C','D'],'B':['E'],'C':['D','F'],'D':['B','E','G'],'E':['D'],'F':['D','G'],'G':['E']}

In [17]:
g = DirectedGraph(d)

In [21]:
g.search('A','B')

The path from A to B is:
['A', 'B']
['A', 'D', 'B']
['A', 'C', 'D', 'B']
['A', 'C', 'F', 'D', 'B']
['A', 'C', 'F', 'G', 'E', 'D', 'B']


In [22]:
class Vertex:
    __slots__ = '_element'
    def __init__(self,x):
        self._element = x
    def element(self):
        return self._element
    def __hash__(self):
        return hash(id(self))

class Edge:
    __slots__ = '_origin','_destination','_element'
    
    def __init__(self,u,v,x):
        self._origin = u
        self._destination = v
        self._element = x
    
    def endpoints(self):
        return (self._origin,self._destination)
    
    def opposite(self,v):
        return self._destination if v is self._origin else self._origin
    
    def element(self):
        return self._element
    
    def __hash__(self):
        return hash((self._origin,self._destination))

class Graph:
    def __init__(self,directed = False):
        self._outgoing = {}
        self._incoming = {} if directed else self._outgoing
        
    def is_directed(self):
        return self._incoming is not self._outgoing
    
    def vertex_count(self):
        return len(self._outgoing)
    
    def vertices(self):
        return self._outgoing.keys()
    
    def edge_count(self):
        total = sum(len(self._outgoing[v] for v in self._outgoing))
        return total if self.is_directed else total//2
    
    def edges(self):
        result = set()
        for secondary_map in self._outgoing.values():
            result.update(secondary_map.values())
        return result
    
    def get_edge(self,u,v):
        return self._outgoing[u],get(v)
    
    def degree(self,v,outgoing = True):
        adj = self._outgoing if outgoing else self._incoming
        return len(adj[v])
    
    def incident_edges(self,v,outgoing = True):
        adj = self._outgoing if outgoing else self._incoming
        for edge in adj[v].values():
            yield edge
            
    def insert_vertex(self,x=None):
        v = self.Vertex()
        self._outgoing[v] = {}
        if self.is_directed():
            self._incoming[v] = {}
        return v
    
    def insert_edge(self,u,v,x=None):
        e = self.Edge(u,v,x)
        self._outgoing[u][v] = e
        self._incoming[v][u] = e
    
    #深度优先搜索
    def DFS(g,u,discovered):
        for e in g.incident_edges(u):
            v = e.opposite(u)
            if v not in discovered:
                discovered[v] = e
                DFS(g,v,discovered)
    
    #广度优先搜索
    def BFS(g,s,discovered):
        level = [s]
        while len(level) > 0:
            next_level = []
            for u in level:
                for e in g.incident_edges(u):
                    v = e.opposite(u)
                    if v not in discovered:
                        discovered[v] = e
                        next_level.append(v)
            level = next_level

## 拓扑排序

In [ ]:
def topological_sort(g):
    topo = []
    ready = []
    incount = {}
    for u in g.vertices():
        incount[u] = g.degree(u,False)
        if incount[u] == 0:
            ready.append(u)
    while len(ready) > 0:
        u = ready.pop()
        topo.append(u)
        for e in g.incident_edges(u):
            v = e.opposite(u)
            incount[v] -= 1
            if incount[v] == 0:
                ready.append(v)
    return topo

### 练习题：岛屿的数量

给定一个由 '1'（陆地）和 '0'（水）组成的的二维网格，计算岛屿的数量。一个岛被水包围，并且它是通过水平方向或垂直方向上相邻的陆地连接而成的。你可以假设网格的四个边均被水包围。

示例 1:

输入:
11110
11010
11000
00000

输出: 1


示例 2:

输入:
11000
11000
00100
00011

输出: 3

In [25]:
class Solution(object):
    def numIslands(self, grid):
        """
        :type grid: List[List[str]]
        :rtype: int
        """
        def dfs(grid, i, j):
            if not 0 <= i < len(grid) or not 0 <= j < len(grid[0]) or grid[i][j] == '0': return
            grid[i][j] = '0'
            dfs(grid, i + 1, j)
            dfs(grid, i, j + 1)
            dfs(grid, i - 1, j)
            dfs(grid, i, j - 1)
        count = 0
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == '1':
                    dfs(grid, i, j)
                    count += 1
        return count
    
    #对于出现1的位置，记一次数，同时将四周的1变为0